In [3]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import csv

In [41]:
# Lectura de datos
nodes = pd.read_csv("../datos/musae_git_target.csv")
atributeDegreeCentrality = pd.read_csv("../metricas/degree_centrality.csv")
atributeDegreeCentralitysorted = atributeDegreeCentrality.sort_values(by='column_name', ascending=True)
atributeClosenessCentrality = pd.read_csv("../metricas/closeness_centrality.csv")
atributeBetweennessCentrality = pd.read_csv("../metricas/betweenness_centrality.csv")
atributeClusteringCoefficient = pd.read_csv("../metricas/clustering_coefficient.csv")
atributeGeneralizedDegree = pd.read_csv("../metricas/generalized_degree.csv")
atributeTriangles = pd.read_csv("../metricas/triangles.csv")
atributeGreedyModularityCommunities = pd.read_csv("../metricas/greedy_modularity_communities.csv")
atributeCoreNumber = pd.read_csv("../metricas/core_number.csv")
atributeAsynLpaCommunities = pd.read_csv("../metricas/asyn_lpa_communities.csv")


In [62]:
# Renombrar columnas para evitar conflictos
atributeDegreeCentrality.rename(columns={'Degree Centrality': 'degree_centrality'}, inplace=True)
atributeClosenessCentrality.rename(columns={'Closeness Centrality': 'closeness_centrality'}, inplace=True)
atributeBetweennessCentrality.rename(columns={'Betweenness Centrality': 'betweenness_centrality'}, inplace=True)
atributeClusteringCoefficient.rename(columns={'Clustering Coefficient': 'clustering_coefficient'}, inplace=True)
atributeGeneralizedDegree.rename(columns={'Generalized Degree': 'generalized_degree'}, inplace=True)
atributeTriangles.rename(columns={'Triangles': 'triangles'}, inplace=True)
atributeGreedyModularityCommunities.rename(columns={'Greedy Modularity Communities': 'greedy_modularity_communities'}, inplace=True)
atributeCoreNumber.rename(columns={'Core Number': 'core_number'}, inplace=True)
atributeAsynLpaCommunities.rename(columns={'Asyn Lpa Communities': 'asyn_lpa_communities'}, inplace=True)

# Añadir columnas directamente al DataFrame nodes
nodes = pd.concat([nodes, atributeDegreeCentrality, atributeClosenessCentrality,
                   atributeBetweennessCentrality, atributeClusteringCoefficient,
                   atributeGeneralizedDegree, atributeTriangles,
                   atributeGreedyModularityCommunities, atributeCoreNumber,
                   atributeAsynLpaCommunities], axis=1)

# Eliminar las tres primeras columnas
nodes = nodes.loc[:, ~nodes.columns.duplicated()]

# Ordenar el DataFrame por la columna 'id'
nodes_sorted = nodes.sort_values(by='id')

nodes.head()


,id,name,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,generalized_degree,triangles,Community,Core number
0,0.0,Eiryyy,0.0,0.000027,0.275005,0.000000e+00,0.000000,Counter({0: 1}),0.0,0,1.0
1,1.0,shawflying,0.0,0.000212,0.294956,1.149733e-06,0.178571,"Counter({0: 3, 1: 2, 3: 2, 2: 1})",5.0,1,6.0
2,2.0,JpMCarrilho,1.0,0.000027,0.261845,0.000000e+00,0.000000,Counter({0: 1}),0.0,0,1.0
3,3.0,SuhwanCha,0.0,0.000133,0.278718,5.316292e-05,0.000000,Counter({0: 5}),0.0,0,4.0
4,4.0,sunilangadi2,1.0,0.000053,0.243084,6.134318e-09,0.000000,Counter({0: 2}),0.0,1,2.0


In [64]:
# Obtener la fila correspondiente al ID específico (por ejemplo, ID = 0)
id_especifico = 6
fila_id_especifico = nodes_sorted.loc[nodes_sorted['id'] == id_especifico]

# Mostrar la fila
print(fila_id_especifico)

    id   name  ml_target  degree_centrality  closeness_centrality  \
6  6.0  sfate        0.0           0.000159              0.372244   

   betweenness_centrality  clustering_coefficient  \
6                0.000002                0.333333   

                  generalized_degree  triangles  Community  Core number  
6  Counter({2: 2, 1: 2, 0: 1, 4: 1})        5.0          1          6.0  


In [ ]:
#Tablas con todos los atributos sin los de conectividad



In [ ]:
#Tablas con todos los atributos sin los de centralidad



In [ ]:
#Tablas con todos los atributos sin los de agrupamiento



In [ ]:
#Tablas con todos los atributos sin los de comunidades



In [ ]:
#Tablas con todos los atributos sin los de nucleos

